# レッスン 01  Predict-Next-Token

このチュートリアルでは、言語モデルの基本メカニズムである「次のトークン予測」（Predict-Next-Token）を深く理解していきます。ゼロから文字レベルの言語モデルを構築し、シェイクスピアの文体を学習させます。この実践プロジェクトを通じて、ニューラルネットワークがどのようにテキストパターンを学習し、特定のスタイルのテキストを生成するかを体験できます。

## Section 1 シェイクスピアテキストデータセット
言語モデルの構築を始める前に、まずトレーニングデータを準備する必要があります。ここでは古典的な「Tiny Shakespeare」データセット——シェイクスピアの全作品を含むテキストコレクションを使用します。

### なぜこのデータセットを選ぶのか？

* 独特なテキストスタイル：シェイクスピアの古英語の文体は特徴的で、モデルの学習効果を観察しやすい
* 適度なデータ量：約1MBのテキストで、高速なトレーニングと実験に適している
* 古典的な教育事例：言語モデル入門チュートリアルで広く使用されている

### データセットのダウンロード

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-10-08 08:32:20--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-10-08 08:32:20 (29.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



このコマンドでGitHubからinput.txtファイルを現在のディレクトリにダウンロードします。

### テキストデータの読み込み

このコードは、テキストファイルを読み込み、データセットの規模を確認する方法を示しています。

In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


with open() ステートメント

* with はPythonのコンテキストマネージャーで、ファイルを自動的に閉じ、リソースリークを防ぎます
* エラーが発生しても、ファイルは正しく閉じられます
* "r": 読み取りモード（read mode）

### テキストの冒頭を確認
テキストの冒頭を確認任意のデータセットを処理する前に、実際の内容をプレビューすることは非常に重要なステップです。

In [3]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


## Section 2 Pytorchをインストールする

PyTorchをインストールする前に、コンピュータのGPU構成を確認する必要があります。ニューラルネットワークのトレーニング時、GPUは計算速度を大幅に向上させます（通常CPUの100倍以上速い）。

### ステップ1：GPUとCUDAバージョンの確認

ターミナルまたはJupyter Notebookで以下のコマンドを実行します：

In [4]:
!nvidia-smi

Wed Oct  8 08:32:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.02              Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060        On  |   00000000:01:00.0  On |                  N/A |
| 30%   40C    P8             N/A /  115W |    1636MiB /   8188MiB |     49%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### コマンドの説明

nvidia-smi (NVIDIA System Management Interface)

* NVIDIAが公式に提供するGPU監視ツール

* GPUモデル、ドライババージョン、CUDAバージョン、VRAMの使用状況などの重要な情報を表示

### 出力例の解説

最初の行：ドライバとCUDA情報

* NVIDIA-SMI 580.65.06：NVIDIAドライババージョン
* Driver Version: 580.65.06：ドライバプログラムバージョン
* CUDA Version: 13.0：最重要！ サポートされる最高CUDAバージョン

GPU詳細情報（GPU毎に1行）

* GPU 0, 1：2枚のGPUが検出されました
* NVIDIA H100 80GB HBM3：GPUモデル
* 26C：現在の温度26℃
* 69W / 700W：現在の消費電力/最大消費電力
* 4MiB / 81559MiB：VRAM使用状況 - 使用中4MB / 合計約80GB
* 0%：GPU利用率

### ステップ2: PyTorchのインストール

PyTorch公式サイトのインストールページにアクセス https://pytorch.org/

環境に応じて適切な設定を選択します：

![install_pytorch](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/install_pytorch.png?raw=true)

### インストールの確認

インストール完了後、以下のコードを実行して確認：

In [5]:
import torch

# PyTorchバージョンの確認
print(f"PyTorchバージョン: {torch.__version__}")

# CUDAが利用可能か確認
print(f"CUDAが利用可能: {torch.cuda.is_available()}")

# CUDAが利用可能な場合、GPU情報を表示
if torch.cuda.is_available():
    print(f"利用可能なGPU数: {torch.cuda.device_count()}")
    print(f"現在のGPU: {torch.cuda.get_device_name(0)}")

PyTorchバージョン: 2.8.0+cu129
CUDAが利用可能: True
利用可能なGPU数: 1
現在のGPU: NVIDIA GeForce RTX 4060


## Section 3 Predict-Next-Token

Predict-Next-Token（次のトークン予測） は、すべての現代的な大規模言語モデル（GPT、Claudeなど）の基礎メカニズムです。この概念を理解することがLLMをマスターする鍵となります。

LLM(例えばGPT)が実際に行っていることは？GPTの本質は最適な次の文字を一文字ずつ探索することです。具体例で理解しましょう：

![LLM1](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/llm1.png?raw=true)

例：「富士山」を生成する

ステップ1：
入力："日本の一番有名な山は？"
GPTモデル予測 → 出力："富"

ステップ2：
入力："日本の一番有名な山は？富"
GPTモデル予測 → 出力："士"

ステップ3：
入力："日本の一番有名な山は？富士"
GPTモデル予測 → 出力："山"

ステップ4：
入力："日本の一番有名な山は富士山です。"
GPTモデル予測 → 出力："(END)" （生成終了）

このプロセスを自己回帰生成（Autoregressive Generation）と呼びます：毎回一文字を予測し、その予測結果を入力に追加して、次を予測し続けます。

### 確率分布：モデルの「思考」プロセス

LLM(GPT)は単純に一文字を出力するのではなく、可能なすべての文字に対して確率を計算します：

![LLM2](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/llm2.png?raw=true)

重要なポイント：
* モデルはすべての可能な文字の確率を計算
* 最も高い確率の文字が必ずしも選ばれるわけではない（これがAIに「創造性」がある理由）
* 温度パラメータで選択のランダム性を制御可能

### 「嘘つきの可能性があり」

* モデルは統計的なパターンに基づいて次の文字を予測するだけ
* 確率が低いオプションも選ばれる可能性がある
* モデルは事実を「理解」せず、トレーニングデータのパターンを模倣するだけ

入力："日本の一番有名な山は？富"

可能な出力：
✓ "士山" (正しい、確率60%)
✗ "山県" (間違い、でも確率3%で選ばれる可能性あり)

## Section 4 シンプルなシェイクスピアLLMの実装する

### 語彙表（Vocabulary）の構築

言語モデルをトレーニングする前に、データにどの文字が含まれているかを知る必要があります。このプロセスを語彙表の構築と呼びます。

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


set() は集合を作成し、重複要素を自動的に削除
シェイクスピアテキストの場合、出現したすべての異なる文字を抽出

### 文字を数値に変換

ニューラルネットワークは数値しか処理できないため、文字↔数値の双方向マッピングを構築する必要があります。このコードはエンコーダーとデコーダーを実装しています。

In [7]:
# 文字から整数へのマッピングを作成
# enumerate()で各文字にインデックス（0, 1, 2...）を割り当てる
stoi = {ch: i for i, ch in enumerate(chars)}  # string to integer: 文字→数値の辞書

# 整数から文字へのマッピングを作成（stoiの逆マッピング）
itos = {i: ch for i, ch in enumerate(chars)}  # integer to string: 数値→文字の辞書

# エンコーダー: 文字列を受け取り、整数のリストを出力
# 例: "hi" → [43, 44]
encode = lambda s: [stoi[c] for c in s]

# デコーダー: 整数のリストを受け取り、文字列を出力  
# 例: [43, 44] → "hi"
decode = lambda l: "".join([itos[i] for i in l])

# エンコードのテスト
encoded_hii_there = encode("hii there")
print(encoded_hii_there)  # 数値リストが表示される

# デコードのテスト（元の文字列に戻る）
print(decode(encoded_hii_there))  # "hii there" が表示される

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


encodeはLAMBDA式から関数に変換：

In [8]:
def encode(s):
    """
    文字列sを数値リストに変換
    引数: s (str) - エンコードする文字列
    戻り値: list[int] - 数値のリスト
    """
    result = []  # 結果を格納するリスト
    for c in s:  # 文字列の各文字をループ
        result.append(stoi[c])  # 文字を数値に変換して追加
    return result

### 練習問題1 decode関数を実装してください。ヒント: encodeの逆の処理を行います
* 整数のリストを受け取る
* 各整数をitosで文字に変換
* 最後に文字を結合して文字列にする

In [9]:
# def decode(s):
    # pass

### データをPyTorchテンソルに変換

エンコード済みのテキストデータをPyTorchが処理できる形式——テンソル（Tensor）に変換します。

In [10]:
# データセット全体をエンコードしてtorch.Tensorに格納する
import torch  # PyTorchをインポート

# テキスト全体を数値に変換し、PyTorchのテンソルに変換
# dtype=torch.long は64ビット整数型を指定（文字IDを格納するため）
data = torch.tensor(encode(text), dtype=torch.long)

# テンソルの形状とデータ型を表示
print(data.shape, data.dtype)  # 例: torch.Size([1115394]) torch.int64

# 最初の200文字分の数値を表示
print(data[:200])  # tensor([18, 47, 56, 57, ...]) のような数値列が表示される

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


### テンソル（Tensor）とは？

テンソルはPyTorchの中核データ構造で、以下のように理解できます：

* 0次元テンソル：スカラー（単一の数値）→ 5
* 1次元テンソル：ベクトル（配列）→ [1, 2, 3, 4]
* 2次元テンソル：行列 → [[1,2], [3,4]]
* 3次元以上：高次元配列

我々のデータは1次元テンソルで、111万文字のIDを含みます。

### データセット分割：トレーニングセットと検証セット

In [11]:
# データセットを訓練用と検証用に分割
n = int(0.9 * len(data))  # 最初の90%を訓練用、残り10%を検証用に
train_data = data[:n]      # 訓練データ：最初の90%
val_data = data[n:]        # 検証データ：残りの10%

### トレーニングサンプルの作成：コンテキストとターゲット


In [12]:
# コンテキストウィンドウのサイズを定義
block_size = 8  # 一度に見る文字数（コンテキスト長）

# 訓練サンプルを作成：入力xと目標yをずらして切り出す
x = train_data[:block_size]          # 入力：最初の8文字
y = train_data[1 : block_size + 1]   # 目標：1文字ずらした8文字

# すべての可能なコンテキスト長でトレーニングサンプルを表示
for t in range(block_size):
    context = x[: t + 1]  # コンテキスト：長さ1からblock_sizeまで増やす
    target = y[t]         # ターゲット：次の文字
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


* block_sizeはモデルが一度に見るコンテキストの文字数を定義
* 同じテキストから複数のトレーニングサンプル（長さ1〜block_size）を生成し、訓練効率を向上
* 各サンプルは「前の文脈から次の文字を予測」という形式

### バッチデータローダー

複数のトレーニングサンプルを並列処理

In [13]:
# 再現性のため乱数シードを固定
torch.manual_seed(1337)

# ハイパーパラメータの設定
batch_size = 4  # 並列処理するシーケンスの数
block_size = 8  # 予測のための最大コンテキスト長


def get_batch(split):
    """
    訓練用または検証用のミニバッチを生成
    引数: split - "train" または "val"
    戻り値: (x, y) - 入力テンソルとターゲットテンソル
    """
    # データセットを選択（訓練用または検証用）
    data = train_data if split == "train" else val_data
    
    # ランダムな開始位置を生成（batch_size個）
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    # 入力x: 各開始位置からblock_size分の文字を取得
    x = torch.stack([data[i : i + block_size] for i in ix])
    
    # ターゲットy: xより1文字先にずらしたデータ
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    
    return x, y


# バッチデータを取得
xb, yb = get_batch("train")

print("inputs:")
print(xb.shape)  # 形状: [batch_size, block_size]
print(xb)

print("targets:")
print(yb.shape)  # 形状: [batch_size, block_size]
print(yb)

print("----")

# バッチ内のすべてのトレーニングサンプルを表示
for b in range(batch_size):      # バッチ次元をループ
    for t in range(block_size):  # 時間次元をループ
        context = xb[b, : t + 1]  # コンテキスト（長さ1〜block_size）
        target = yb[b, t]         # 次の文字
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

* batch_size=4：一度に4つの独立したテキストシーケンスを並列処理し、訓練効率を向上
* ランダムサンプリング：データセットからランダムに開始位置を選択し、モデルが固定順序を記憶するのを防ぐ
* torch.stack：複数の1Dテンソルを2Dテンソル [batch_size, block_size] にスタック
* 1つのバッチには batch_size × block_size = 4 × 8 = 32 個のトレーニングサンプルが含まれる

### 最もシンプルな言語モデルを構築

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# 再現性のため乱数シードを固定
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    """
    Bigramモデル: 直前の1文字だけを見て次の文字を予測する最もシンプルなLLM
    """

    def __init__(self, vocab_size):
        super().__init__()
        # 各トークンが次のトークンのロジット（確率の元）を直接読み取るルックアップテーブル
        # 形状: [vocab_size, vocab_size] - 各文字が次の各文字への「好み」を持つ
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        """
        順伝播: 入力から予測を生成
        idx: (B, T) - バッチサイズB、シーケンス長Tの入力インデックス
        targets: (B, T) - 正解ラベル（訓練時のみ）
        """
        # idxとtargetsはどちらも (B,T) の整数テンソル
        # ルックアップテーブルから各トークンの予測を取得
        logits = self.token_embedding_table(idx)  # (B, T, C) - C=vocab_size
        
        # 損失の計算（訓練時のみ）
        if targets is None:
            loss = None
        else:
            # PyTorchのcross_entropyは2D入力を期待するため形状を変換
            B, T, C = logits.shape
            logits = logits.view(B*T, C)      # (B*T, C) に平坦化
            targets = targets.view(B*T)       # (B*T) に平坦化
            # クロスエントロピー損失: 予測と正解の差を測定
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        """
        テキスト生成: 現在のコンテキストから新しいトークンを生成
        idx: (B, T) - 現在のコンテキストのインデックス配列
        max_new_tokens: 生成する新しいトークンの数
        """
        # max_new_tokens分だけ繰り返す
        for _ in range(max_new_tokens):
            # 予測を取得
            logits, loss = self(idx)
            
            # 最後のタイムステップのみに注目（Bigramは直前の1文字だけを使用）
            logits = logits[:, -1, :]  # (B, C) になる
            
            # ソフトマックスを適用して確率分布に変換
            probs = F.softmax(logits, dim=-1)  # (B, C)
            
            # 確率分布からサンプリング（ランダムに選択）
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            
            # サンプリングしたインデックスを実行中のシーケンスに追加
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        
        return idx

# モデルのインスタンス化
m = BigramLanguageModel(vocab_size)

# 訓練バッチで順伝播を実行
logits, loss = m(xb, yb)
print(logits.shape)  # 予測の形状を表示
print(loss)          # 初期損失を表示（訓練前なのでランダム）

# テキスト生成のテスト
# torch.zeros((1, 1), dtype=torch.long) は改行文字（インデックス0）から開始
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


## モデルのトレーニングとテキスト生成

モデルをトレーニングしてシェイクスピア風テキストを生成

In [15]:
# PyTorchのオプティマイザー（最適化アルゴリズム）を作成
# AdamW: 学習率を自動調整する高性能な最適化手法
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32  # バッチサイズを32に設定

# トレーニングループ: 10000ステップ繰り返す
for steps in range(10000):  # より良い結果のためステップ数を増やす

    # 訓練データからバッチをサンプリング
    xb, yb = get_batch("train")

    # 損失を評価（予測と正解の差を計算）
    logits, loss = m(xb, yb)
    
    # 勾配をゼロにリセット（前回の勾配を消去）
    optimizer.zero_grad(set_to_none=True)
    
    # 誤差逆伝播: 損失から勾配を計算
    loss.backward()
    
    # パラメータを更新（勾配降下法）
    optimizer.step()

# 最終的な損失を表示
print(loss.item())

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

2.382369041442871

lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


## 次回の予告

In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 32  # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 1000
learning_rate = 1e-3
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

print("Running on " + device)

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [
    stoi[c] for c in s
]  # encoder: take a string, output a list of integers
decode = lambda l: "".join(
    [itos[i] for i in l]
)  # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B,T,C)
        q = self.query(x)  # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()) / 1e6, "M parameters")

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


Running on cuda
0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 1000: train loss 2.1005, val loss 2.1338
step 2000: train loss 1.8832, val loss 1.9785
step 3000: train loss 1.7747, val loss 1.9133
step 4000: train loss 1.7249, val loss 1.8786
step 4999: train loss 1.6673, val loss 1.8170


YROMBERLA:
Ock, and is the tombanded boay uservater--

MARCILLAUUE:
Wart he usque, toubart that ane away, my feand' to zoloug
Yourselvefuit here to the will,
Which ensend, will is the overs, and if the honourable. Ahland me like us, oncriby: but have the tybunle.

SICINIUS:
Alay sweep
Is would thake only so whrowerings to them,
His hands in he poor of his but to dangert,
If so;
Angies must with aled atters
Marry, I home, and some strebled:
Shone Was happest hoights knear teyour-busich
To for his neever kind my lose and gand me
That he see--'

NORUS:
And madam? whock blive with welcome,
Thou counfepy to the might. 
ELAURET:
For injursored and be tooget, if parms
Wouch in meedy so